[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/idealo/image-super-resolution/blob/master/notebooks/ISR_Prediction_Tutorial.ipynb)

# Install ISR

In [ ]:
!pip uninstall ISR

# Predict

## Get the pre-trained weights and data

(for more options on getting you data on Colab notebooks visit https://colab.research.google.com/notebooks/io.ipynb)

Get the weights with



In [ ]:
!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rdn-C6-D20-G64-G064-x2/ArtefactCancelling/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5
!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rdn-C6-D20-G64-G064-x2/PSNR-driven/rdn-C6-D20-G64-G064-x2_PSNR_epoch086.hdf5
!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rdn-C3-D10-G64-G064-x2/PSNR-driven/rdn-C3-D10-G64-G064-x2_PSNR_epoch134.hdf5
!wget https://media.githubusercontent.com/media/idealo/image-super-resolution/master/weights/sample_weights/rrdn-C4-D3-G32-G032-T10-x4/Perceptual/rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf5
!mkdir weights
!mv *.hdf5 weights

Download a sample image, in this case

In [ ]:
!wget http://images.math.cnrs.fr/IMG/png/section8-image.png
!mkdir -p data/input/test_images
!mv *.png data/input/test_images

Load the image with PIL

In [ ]:
import numpy as np,os,sys
from PIL import Image
#dir = os.path.join(os.path.dirname(os.getcwd()),os.path.dirname('data\\input\\test_images\\'),'section8-image.png')
dir = os.path.join(os.path.dirname(os.getcwd()),os.path.dirname('data\\input\\test_images\\'),'Shematics IO board.png')

print(dir)
img = Image.open(dir)
img

## Get predictions

### Create the model and run prediction
Create the RDN model, for which we provide pre-trained weights, and load them

Choose amongst the available model weights, compare the output if you wish.



In [ ]:
#os.chdir(os.path.join('D:\\','Git Apps\\image-super-resolution'))
print(os.getcwd())
sys.path.append(os.path.join('D:\\','Git Apps\\image-super-resolution'))
print(sys.path)

In [ ]:
from ISR.models import RRDN,RDN

In [ ]:
sess = RRDN.tf.Session()
print(sess)

#### RRDN GANS model

In [ ]:
rrdn = RRDN(arch_params={'C': 4, 'D':3, 'G':32, 'G0':32, 'x':4, 'T': 10})
dir = os.path.join(os.path.dirname(os.getcwd()),\
                   'weights','sample_weights','rrdn-C4-D3-G32-G032-T10-x4-GANS','rrdn-C4-D3-G32-G032-T10-x4_epoch299.hdf5')
print(dir)
rrdn.model.load_weights(dir)


In [ ]:
print(img.info, img.size)
a4im = Image.new('RGB',
                 (1376, 1376),   # 1376 pixel
                 (255, 255, 255))  # White
a4im.paste(img, img.getbbox())
print(a4im.info,a4im.size)

In [ ]:
sr_img = rrdn.predict(np.array(a4im),by_patch_of_size=0,padding_size=2,batch_size=2)
Image.fromarray(sr_img)

#### Large RDN model

In [ ]:
rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_PSNR_epoch086.hdf5')

#### Small RDN model

In [ ]:
rdn = RDN(arch_params={'C': 3, 'D':10, 'G':64, 'G0':64, 'x':2})
rdn.model.load_weights('weights/rdn-C3-D10-G64-G064-x2_PSNR_epoch134.hdf5')

#### Large RDN noise cancelling, detail enchancing model

In [ ]:
rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5')

#### Run prediction

In [ ]:
sr_img = rdn.predict(np.array(img))
Image.fromarray(sr_img)

### Usecase: upscaling noisy images

Now, for science, let's make it harder for the networks.

We compress the image into the jpeg format to introduce compression artefact and lose some information.

We will compare:
- the baseline bicubic scaling
- the basic model - Add Hyperlink
- a model trained to remove noise using perceptual loss with deep features and GANs training

So let's first  compress the image

In [ ]:
img.save('data/input/test_images/compressed.jpeg','JPEG', dpi=[300, 300], quality=50)
compressed_img = Image.open('data/input/test_images/compressed.jpeg')

compressed_img

(open the image in a new tab and zoom in to inspect it)

#### Baseline
Bicubic scaling

In [ ]:
compressed_img.resize(size=(compressed_img.size[0]*2, compressed_img.size[1]*2), resample=Image.BICUBIC)

#### Large RDN model (PSNR trained)

In [ ]:
rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_PSNR_epoch086.hdf5')
sr_img = rdn.predict(np.array(compressed_img))
Image.fromarray(sr_img)

#### Small RDN model  (PSNR trained)

In [ ]:
rdn = RDN(arch_params={'C': 3, 'D':10, 'G':64, 'G0':64, 'x':2})
rdn.model.load_weights('weights/rdn-C3-D10-G64-G064-x2_PSNR_epoch134.hdf5')
sr_img = rdn.predict(np.array(compressed_img))
Image.fromarray(sr_img)

#### Large RDN noise cancelling, detail enhancing model

In [ ]:
rdn = RDN(arch_params={'C': 6, 'D':20, 'G':64, 'G0':64, 'x':2})
rdn.model.load_weights('weights/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5')
sr_img = rdn.predict(np.array(compressed_img))
Image.fromarray(sr_img)

### Predictor Class
You can also use the predictor class to run the model on entire folders:

In [ ]:
from ISR.predict import Predictor
!mkdir -p data/output
predictor = Predictor(input_dir='data/input/test_images/')
predictor.get_predictions(model=rdn, weights_path='weights/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5')